In [3]:
# Student Name : Markus Proesch
# Cohort       : 4


################################################################################
# Import Packages
################################################################################

import pandas as pd                                      # data science essentials
import matplotlib.pyplot as plt                          # data visualization
import seaborn as sns                                    # enhanced data visualization
import statsmodels.formula.api as smf                    # explanatory model 
from sklearn.model_selection import train_test_split     # divides dataset into a train and test set
from sklearn.linear_model import LogisticRegression      # Logistic Regression
from sklearn.metrics import confusion_matrix             # confusion matrix
from sklearn.metrics import roc_auc_score                # Calculating the ROC and AUC
from sklearn.neighbors import KNeighborsClassifier       # KNN for classification
from sklearn.preprocessing import StandardScaler         # Standardizing values
from sklearn.tree import DecisionTreeClassifier          # classification trees
from sklearn.ensemble import GradientBoostingClassifier  # Gradient Booster for classification
from sklearn.ensemble import RandomForestClassifier      # Random Forest for classification   
from sklearn.model_selection import GridSearchCV         # hyperparameter tuning
from sklearn.metrics import make_scorer                  # customizable scorer


################################################################################
# Load Data
################################################################################


original_df     = pd.read_excel('Apprentice Chef Dataset.xlsx')


################################################################################
# Feature Engineering, Variable Selection and (optional) Dataset Standardization
################################################################################

# Add Avg. Price per Meal variable
original_df['AVG_PRICE_PER_MEAL'] = original_df['REVENUE']/original_df['TOTAL_MEALS_ORDERED']


# Flagging all observations/customers with no input in FAMILY_NAME
for col in original_df:

        if original_df[col].isnull().astype(int).sum() > 0:
            original_df['mv_'+col] = original_df[col].isnull().astype(int)

# Filled NA in FAMILY_NAME with same name as in FIRST_NAME since that seems to be the way to do it
original_df['FAMILY_NAME'] = original_df['FAMILY_NAME'].fillna(original_df['FIRST_NAME'])

# Drop the flagged missing values in FAMILY_NAME as they are replaced with FIRST_NAME
original_df = original_df.drop(columns = 'mv_FAMILY_NAME')            
            

# Dummie variables from the email domain.
# Dataset has to be a DataFrame for .iterrows() to work
original_df_email       = pd.DataFrame(original_df['EMAIL'])

placeholder_lst  = []

for index, col in original_df_email.iterrows():
    split_email  = original_df_email.loc[index, 'EMAIL'].split(sep = '@')
    
    placeholder_lst.append(split_email)
    
email_df         = pd.DataFrame(placeholder_lst)
email_df.columns = ['name', 'domain']

# Domain groups
personal_domain     = ['@gmail.com', '@yahoo.com','@protonmail.com']
professional_domain = ['@mmm.com', '@amex.com','@apple.com',
                      '@boeing.com','@caterpillar.com',
                      '@chevron.com','@cisco.com','@cocacola.com',
                      '@disney.com','@dupont.com','@exxon.com',
                      '@ge.org','@goldmansacs.com','@homedepot.com',
                      '@ibm.com','@intel.com','@jnj.com',
                      '@jpmorgan.com','@mcdonalds.com','@merck.com',
                      '@microsoft.com','@nike.com','@pfizer.com',
                      '@pg.com','@travelers.com','@unitedtech.com',
                      '@unitedhealth.com','@verizon.com','@visa.com',
                      '@walmart.com']
junk_domain         = ['@me.com', '@aol.com', '@hotmail.com', '@live.com',
                       '@msn.com','@passport.com']

# For loop categorising the different email domains
placeholder_lst = []

for domain in email_df['domain']:
    
    if '@' + domain in personal_domain:
        placeholder_lst.append('PERSONAL_DOMAIN')
    elif '@' + domain in professional_domain:
        placeholder_lst.append('PROFESSIONAL_DOMAIN')
    else:
        placeholder_lst.append('JUNK_DOMAIN')
        
# make the columns into a series to append it to original dataset        
email_df['DOMAIN_GROUP'] = pd.Series(placeholder_lst)

# Add the domain categories column to the original dataset 
original_df['DOMAIN'] = email_df['DOMAIN_GROUP']

# Get dummies from the domain variable and drop the original column
one_hot_DOMAIN = pd.get_dummies(original_df['DOMAIN'])

# Remove the old and add the 3 new dummy variables (as columns)
original_df           = original_df.drop('DOMAIN', axis = 1)
original_df           = original_df.join([one_hot_DOMAIN])


# Adding variable, counting the number of names in NAME column

def text_split_feature(col, df, sep=' ', new_col_name=None):
    """
Splits values in a string Series (as part of a DataFrame) and sums the number
of resulting items. Automatically appends summed column to original DataFrame.

PARAMETERS
----------
col          : column to split
df           : DataFrame where column is located
sep          : string sequence to split by, default ' '
new_col_name : name of new column after summing split, default
               'number_of_names'
"""
    
    original_df[new_col_name] = 0
    
    
    for index, val in original_df.iterrows():
        original_df.loc[index, new_col_name] = len(original_df.loc[index, col].split(sep = ' '))
        
text_split_feature(col = 'NAME', df = original_df, new_col_name = 'NUMBER_NAMES' )

# Flagging variable where FIRST NAME is the same as FAMILY NAME
placeholder_lst = []

for row,col in original_df.iterrows():
    if original_df.loc[row,'FIRST_NAME'] == original_df.loc[row,'FAMILY_NAME']:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

# Adding the placeholder to the SAME_NAME column and drop the old column
original_df['SAME_NAME'] = pd.Series(placeholder_lst)


# Making ATTENDED_MASTER_CLASS a binary variable
# 1 = attended 1 or more classes, 0 = did not attend a class
placeholder_lst = []

for row,col in original_df.iterrows():
    if original_df.loc[row,'MASTER_CLASSES_ATTENDED'] >= 1:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

# Adding the placeholder to the ATTENDED_MASTER_CLASS column and drop the old column
original_df['ATTENDED_MASTER_CLASS'] = pd.Series(placeholder_lst)
original_df = original_df.drop(columns = 'MASTER_CLASSES_ATTENDED')


# Flagging NOBLE people from the NAME column
# 1 = NOBLE, 0 = Not NOBLE
placeholder_lst = []

for row,pattern in original_df.iterrows():
    if ' of ' in original_df.loc[row,'NAME'] or \
    'lord' in original_df.loc[row,'NAME'] or \
    'Lord' in original_df.loc[row,'NAME'] or \
    ' mo ' in original_df.loc[row,'NAME'] or \
    ' zo ' in original_df.loc[row,'NAME'] or \
    ' Mo ' in original_df.loc[row,'NAME'] or \
    'Knight' in original_df.loc[row, 'NAME'] or \
    'knight'in original_df.loc[row, 'NAME']:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

original_df['NOBLE'] = pd.Series(placeholder_lst)


# Outliers thresholds determined based on the histograms and scatterplots
revenue_hi                    = 6500
total_meals_hi                = 320
unique_meals_hi               = 12
contact_w_customer_service_hi = 12
avg_time_per_site_hi          = 400
cancel_before_noon_hi         = 7
late_deliveries_hi            = 15
avg_prep_video_hi             = 350
total_photoes_hi              = 900
avg_meal_price                = 120
follow_rec_pct_hi             = 30
follow_rec_pct_lo             = 1


# FOLLOWED_RECOMMENDATIONS_PCT
original_df['out_FOLLOWED_RECOMMENDATIONS_PCT']  = 0
condition_hi = original_df.loc[0:,'out_FOLLOWED_RECOMMENDATIONS_PCT'][original_df['FOLLOWED_RECOMMENDATIONS_PCT'] 
                                                                      > follow_rec_pct_hi]
condition_lo = original_df.loc[0:,'out_FOLLOWED_RECOMMENDATIONS_PCT'][original_df['FOLLOWED_RECOMMENDATIONS_PCT'] 
                                                                      < follow_rec_pct_lo]


original_df['out_FOLLOWED_RECOMMENDATIONS_PCT'].replace(to_replace = condition_hi,
                                                        value      = 1,
                                                        inplace    = True)
original_df['out_FOLLOWED_RECOMMENDATIONS_PCT'].replace(to_replace = condition_lo,
                                                        value      = 1,
                                                        inplace    = True)

# REVENUE
original_df['out_REVENUE']  = 0
condition_hi = original_df.loc[0:,'out_REVENUE'][original_df['REVENUE'] 
                                                          > revenue_hi]

original_df['out_REVENUE'].replace(to_replace = condition_hi,
                                   value      = 1,
                                   inplace    = True)
# TOTAL_MEALS_ORDERED
original_df['out_TOTAL_MEALS_ORDERED']  = 0
condition_hi = original_df.loc[0:,'out_TOTAL_MEALS_ORDERED'][original_df['TOTAL_MEALS_ORDERED'] 
                                                             > total_meals_hi]

original_df['out_TOTAL_MEALS_ORDERED'].replace(to_replace = condition_hi,
                                               value      = 1,
                                               inplace    = True)

# UNIQUE_MEALS_PURCH
original_df['out_UNIQUE_MEALS_PURCH']  = 0
condition_hi = original_df.loc[0:,'out_UNIQUE_MEALS_PURCH'][original_df['UNIQUE_MEALS_PURCH'] 
                                                            > unique_meals_hi]

original_df['out_UNIQUE_MEALS_PURCH'].replace(to_replace = condition_hi,
                                              value      = 1,
                                              inplace    = True)

# CONTACTS_W_CUSTOMER_SERVICE
original_df['out_CONTACTS_W_CUSTOMER_SERVICE']  = 0
condition_hi = original_df.loc[0:,'out_CONTACTS_W_CUSTOMER_SERVICE'][original_df['CONTACTS_W_CUSTOMER_SERVICE'] 
                                                                     > contact_w_customer_service_hi]

original_df['out_CONTACTS_W_CUSTOMER_SERVICE'].replace(to_replace = condition_hi,
                                                       value      = 1,
                                                       inplace    = True)

# AVG_TIME_PER_SITE_VISIT
original_df['out_AVG_TIME_PER_SITE_VISIT']  = 0
condition_hi = original_df.loc[0:,'out_AVG_TIME_PER_SITE_VISIT'][original_df['AVG_TIME_PER_SITE_VISIT'] 
                                                                 > avg_time_per_site_hi]

original_df['out_AVG_TIME_PER_SITE_VISIT'].replace(to_replace = condition_hi,
                                                   value      = 1,
                                                   inplace    = True)

# CANCELLATIONS_BEFORE_NOON
original_df['out_CANCELLATIONS_BEFORE_NOON']  = 0
condition_hi = original_df.loc[0:,'out_CANCELLATIONS_BEFORE_NOON'][original_df['CANCELLATIONS_BEFORE_NOON'] 
                                                                   > cancel_before_noon_hi]

original_df['out_CANCELLATIONS_BEFORE_NOON'].replace(to_replace = condition_hi,
                                                     value      = 1,
                                                     inplace    = True)

# LATE_DELIVERIES
original_df['out_LATE_DELIVERIES']  = 0
condition_hi = original_df.loc[0:,'out_LATE_DELIVERIES'][original_df['LATE_DELIVERIES'] 
                                                         > late_deliveries_hi]

original_df['out_LATE_DELIVERIES'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)

# AVG_PREP_VID_TIME
original_df['out_AVG_PREP_VID_TIME']  = 0
condition_hi = original_df.loc[0:,'out_AVG_PREP_VID_TIME'][original_df['AVG_PREP_VID_TIME'] 
                                                          > avg_prep_video_hi]

original_df['out_AVG_PREP_VID_TIME'].replace(to_replace = condition_hi,
                                             value      = 1,
                                             inplace    = True)

# TOTAL_PHOTOS_VIEWED
original_df['out_TOTAL_PHOTOS_VIEWED']  = 0
condition_hi = original_df.loc[0:,'out_TOTAL_PHOTOS_VIEWED'][original_df['TOTAL_PHOTOS_VIEWED'] 
                                                          > total_photoes_hi]

original_df['out_TOTAL_PHOTOS_VIEWED'].replace(to_replace = condition_hi,
                                               value      = 1,
                                               inplace    = True)

# AVG_PRICE_PER_MEAL
original_df['out_AVG_PRICE_PER_MEAL']  = 0
condition_hi = original_df.loc[0:,'out_AVG_PRICE_PER_MEAL'][original_df['AVG_PRICE_PER_MEAL'] 
                                                          > avg_meal_price]

original_df['out_AVG_PRICE_PER_MEAL'].replace(to_replace = condition_hi,
                                              value      = 1,
                                              inplace    = True)


# Dictinary with significant variables and all available variables 
variable_dict = {
    'logit_sig' : ['REVENUE', 'MOBILE_NUMBER','CANCELLATIONS_BEFORE_NOON',
                   'CANCELLATIONS_AFTER_NOON','EARLY_DELIVERIES', 'LATE_DELIVERIES',
                   'FOLLOWED_RECOMMENDATIONS_PCT', 'AVG_CLICKS_PER_VISIT',
                    'JUNK_DOMAIN', 'PROFESSIONAL_DOMAIN', 'NUMBER_NAMES',
                   'SAME_NAME','ATTENDED_MASTER_CLASS', 'out_FOLLOWED_RECOMMENDATIONS_PCT',
                   'NOBLE'],
    
    
   'logit_full' : ['REVENUE', 'TOTAL_MEALS_ORDERED',
                   'UNIQUE_MEALS_PURCH', 'CONTACTS_W_CUSTOMER_SERVICE', 
                   'PRODUCT_CATEGORIES_VIEWED', 'AVG_TIME_PER_SITE_VISIT', 
                   'MOBILE_NUMBER', 'CANCELLATIONS_BEFORE_NOON',
                   'CANCELLATIONS_AFTER_NOON', 'TASTES_AND_PREFERENCES',
                   'MOBILE_LOGINS', 'PC_LOGINS', 'EARLY_DELIVERIES', 
                   'LATE_DELIVERIES', 'PACKAGE_LOCKER', 'REFRIGERATED_LOCKER',
                   'FOLLOWED_RECOMMENDATIONS_PCT', 'AVG_PREP_VID_TIME', 
                   'LARGEST_ORDER_SIZE', 'MEDIAN_MEAL_RATING', 
                   'AVG_CLICKS_PER_VISIT', 'TOTAL_PHOTOS_VIEWED', 
                   'AVG_PRICE_PER_MEAL', 'JUNK_DOMAIN', 'PROFESSIONAL_DOMAIN', 'NUMBER_NAMES', 'SAME_NAME',
                   'NOBLE', 'WEEKLY_PLAN',
                   'out_FOLLOWED_RECOMMENDATIONS_PCT', 'out_REVENUE',
                   'out_TOTAL_MEALS_ORDERED', 'out_UNIQUE_MEALS_PURCH',
                   'out_CONTACTS_W_CUSTOMER_SERVICE', 
                   'out_AVG_TIME_PER_SITE_VISIT', 'out_CANCELLATIONS_BEFORE_NOON',
                   'out_LATE_DELIVERIES', 'out_AVG_PREP_VID_TIME', 
                   'out_TOTAL_PHOTOS_VIEWED', 'out_AVG_PRICE_PER_MEAL']
}

################################################################################
# Train/Test Split
################################################################################

# Divide into a data (with significant variables) and a target dataset
original_df_data   =  original_df.loc[ : , variable_dict['logit_sig']]
original_df_target =  original_df.loc[ : , 'CROSS_SELL_SUCCESS']


# Divide into train and test dataset and stratify on FOLLOWED_RECOMMENDATIONS_PCT 
X_train, X_test, y_train, y_test = train_test_split(
            original_df_data,
            original_df_target,
            random_state = 222,
            test_size    = 0.25,
            stratify     = original_df['FOLLOWED_RECOMMENDATIONS_PCT'])



################################################################################
# Final Model (instantiate, fit, and predict)
################################################################################

# INSTANTIATING a classification 
g_boost = GradientBoostingClassifier(loss = 'deviance',
                                     criterion = 'mae',
                                     learning_rate =  0.1,
                                     n_estimators = 95,
                                     max_features = 3,
                                     random_state  = 222)

# FITTING the training data
g_boost_fit = g_boost.fit(X_train, y_train)

# PREDICTING on test data
g_boost_pred = g_boost_fit.predict(X_test)


################################################################################
# Final Model Score (score)
################################################################################


test_score = roc_auc_score(y_true  = y_test,
                           y_score = g_boost_pred).round(4)

